In [1]:
import numpy as np
import pandas as pd

In [2]:
from my_packages.layers import Dense
from my_packages.policy import Policy
from my_packages.environment import StockEnv
from my_packages.agent import Agent

# 1. Handling Data

In [3]:
api_query = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=1min&month=2025-02&outputsize=full&datatype=csv&apikey=6QP1RCVBQ5TCGE65"

In [4]:
df = pd.read_csv("data/IBM_feb_25.csv")
df.head()

,timestamp,open,high,low,close,volume
0,2025-02-28 19:59:00,252.5,252.5,252.50,252.50,1
1,2025-02-28 19:58:00,252.5,252.5,252.03,252.03,45
2,2025-02-28 19:53:00,252.5,252.5,252.50,252.50,116
3,2025-02-28 19:51:00,252.5,252.5,252.50,252.50,15
4,2025-02-28 19:49:00,252.5,252.5,252.50,252.50,4


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12388 entries, 0 to 12387
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  12388 non-null  object 
 1   open       12388 non-null  float64
 2   high       12388 non-null  float64
 3   low        12388 non-null  float64
 4   close      12388 non-null  float64
 5   volume     12388 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 580.8+ KB


In [6]:
df.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [7]:
# DATA ENGINEER WITH THE DATA I'M SKIPPING THE PART AS OF NOW 
# BE CAREFULL TO CHANGE THE INDEX OF RESPECTIVE DATA IN THE environment.py

In [8]:
df = df.iloc[:,1:]

In [9]:
X = df.to_numpy(dtype = float)

In [10]:
X.shape

(12388, 5)

# 2. Building model

In [11]:
env = StockEnv()
env.set_env_data(X)

In [12]:
policy = Policy([Dense(units=10, activation='relu'),
                 Dense(units=3, activation='linear')],
                loss = 'mse')

In [13]:
policy.episode_train(env)

AttributeError: 'StockEnv' object has no attribute 'get_current_state'